In [6]:
import tensorflow.keras
from homemade_unet import unet_model_3d
from unet_scratch import get_unet

In [7]:
input_shape = (1,64,64,64)
#model = unet_model_3d(input_shape = input_shape, n_labels = 3)
model = get_unet(input_img = input_shape, n_labels = 3, n_filters = 16, batchnorm = True)
#model.summary()

ValueError: Layer conv3d_1 expects 1 inputs, but it received 4 input tensors. Inputs received: [<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=64>, <tf.Tensor: shape=(), dtype=int32, numpy=64>, <tf.Tensor: shape=(), dtype=int32, numpy=64>]

In [3]:
from data import get_train_and_validation_datasets

batch_size = 8
path_images = '/home/jupyter/ai_postgraduate_project/data/raw_dataset/imagesTr/'
path_labels = '/home/jupyter/ai_postgraduate_project/data/raw_dataset/labelsTr/'

train_dataset, validation_dataset, validation_images = get_train_and_validation_datasets(0.2,path_images,path_labels,patch=True,patch_shape=(64,64,64),subsample=20)
train_dataset = train_dataset.shuffle(100).batch(batch_size).prefetch(2)
validation_dataset = validation_dataset.batch(batch_size).prefetch(2)


Subsampling with images: ['pancreas_182.nii.gz', 'pancreas_277.nii.gz', 'pancreas_080.nii.gz', 'pancreas_387.nii.gz', 'pancreas_253.nii.gz', 'pancreas_088.nii.gz', 'pancreas_111.nii.gz', 'pancreas_222.nii.gz', 'pancreas_357.nii.gz', 'pancreas_098.nii.gz', 'pancreas_212.nii.gz', 'pancreas_294.nii.gz', 'pancreas_019.nii.gz', 'pancreas_211.nii.gz', 'pancreas_180.nii.gz', 'pancreas_261.nii.gz', 'pancreas_298.nii.gz', 'pancreas_207.nii.gz', 'pancreas_369.nii.gz', 'pancreas_055.nii.gz']
Number of images for training: 16
Number of images for validation: 4


In [ ]:
for image in train_dataset:
    x,y = image
    print(x.shape)
    print(y.shape)
    break

In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from datetime import datetime

time = datetime.now().strftime('%Y-%m-%d_%H-%M')

model_checkpoint = ModelCheckpoint(
    filepath='/home/jupyter/ai_postgraduate_project/notebooks/cpachon/models/best/' + time + '_model_weights.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True)

reduce_lr_plateau = ReduceLROnPlateau(monitor='val_loss', 
                                      patience=10, 
                                      verbose=1)

tensorboard_callback = TensorBoard(log_dir='runs/' + time,
                                   write_grads=True)

callbacks = [model_checkpoint,reduce_lr_plateau,tensorboard_callback]


In [5]:
epochs = 80
model.fit(train_dataset,
          epochs=epochs,
          validation_data=validation_dataset,
          callbacks = callbacks)

Epoch 1/80
     60/Unknown - 272s 5s/step - loss: 2.4260
Epoch 00001: val_loss improved from inf to 0.92509, saving model to /home/jupyter/ai_postgraduate_project/notebooks/cpachon/models/best/2020-03-14_18-19_model_weights.h5
60/60 [==============================] - 298s 5s/step - loss: 2.4260 - val_loss: 0.9251
Epoch 2/80
46/60 [======================>.......] - ETA: 59s - loss: 1.5368 WARNING:tensorflow:Can save best model only with val_loss available, skipping.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-d90bf0ec6193>", line 5, in <module>
    callbacks = callbacks)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 342, in fit
    total_epochs=epochs)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 128, in run_one_epoch
    batch_outs = execution_function(iterator)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2_utils.py", line 98, in execution_function
    distributed_function(input_fn))
  File "/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/u

KeyboardInterrupt: 

In [ ]:
model.save_weights("models/final/" + time + "_model_weights.h5")